<h3>Import all the required packages</h3>

In [0]:
!pip install pyspellchecker

import nltk
nltk.download('wordnet')
nltk.download('movie_reviews')
nltk.download('stopwords')

     |████████████████████████████████| 1.9MB 3.4MB/s 
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.corpus import movie_reviews
import pandas as pd
import numpy as np
from string import punctuation
import random
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from spellchecker import SpellChecker
from datetime import datetime
import re,string
training_chart=[]

<h3> This procedure will change the dual words to actual words line you'll to You will</h3>

In [0]:
def change_dual_words(word):
    first=''
    second=''
    def get_correct_word(w):
        if w=="sha":
            return("shall")
        elif w=="wo":
            return("will")
        elif w=="y":
            return("you")
        else:
            return(w)
    word=word.strip().strip("'").strip(',').strip('`').strip('[').strip(']').strip('-').strip('_').strip('#').strip('+').strip('/').strip('(').strip(')').lower()
    if word.endswith("n't"):
        first=get_correct_word(word[:word.index("n't")])
        second="not"
    elif word.endswith("'ll"):
        first=get_correct_word(word[:word.index("'ll")])
        second="will"
    elif word.endswith("'ve"):
        first=get_correct_word(word[:word.index("'ve")])
        second="have"
    elif word.endswith("'re"):
        first=get_correct_word(word[:word.index("'re")])
        second="are"
    elif word.endswith("'d"):
        first=get_correct_word(word[:word.index("'d")])
        second="did"
    elif word.endswith("'s"):
        first=get_correct_word(word[:word.index("'s")])
        second="is"
    elif word.endswith("'all"):
        first=get_correct_word(word[:word.index("'all")])
        second="all"
    else:
        first=word
        second="<allclear>"
    return((first,second))

<h3>Clean the lines</h3>

In [0]:
def cleanLine(line):
#     stopword = stopwords.words('english')
    stopword=['aa','aah','aahhs','aahs','(',')',"''",',','``','-','.','4','2','5','9','26','$$','$@^@','&#','&nbsp','&quot','#@%$^','==','>from','>honk','@#$^%','@#%','@$&%','a&m','a&w','a+']
    stopwordList=set(stopword)
    tokenizer = RegexpTokenizer('\w+-+\w+')
    hyphenated_word=tokenizer.tokenize(line)
#     print(hyphenated_word)
    for word in hyphenated_word:
        line=line.replace(word,word.replace('-',' '))
    
    tokenizer = RegexpTokenizer('\w+-\w+')
    hyphenated_word=tokenizer.tokenize(line)
#     print(hyphenated_word)
    for word in hyphenated_word:
        line=line.replace(word,word.replace('-',' '))
        
    tokenizer = RegexpTokenizer('\w+_\w+')
    hyphenated_word=tokenizer.tokenize(line)
#     print(hyphenated_word)
    for word in hyphenated_word:
        line=line.replace(word,word.replace('_',' '))
        
    tokenizer = RegexpTokenizer('\w+/\w+')
    hyphenated_word=tokenizer.tokenize(line)
#     if len(hyphenated_word)>0:print(hyphenated_word)
    for word in hyphenated_word:
        line=line.replace(word,word.replace('/',' '))
        
    tokenizer = RegexpTokenizer('\w+/+\w+')
    hyphenated_word=tokenizer.tokenize(line)
#     if len(hyphenated_word)>0:print(hyphenated_word)
    for word in hyphenated_word:
        line=line.replace(word,word.replace('/',' '))
    
    tokenizer = RegexpTokenizer('\w+/\'\w+')
    hyphenated_word=tokenizer.tokenize(line)
#     if len(hyphenated_word)>0:print(hyphenated_word)
    for word in hyphenated_word:
        line=line.replace(word,word.replace('/\'',' '))
        
#     tokenizer = RegexpTokenizer('\w+\'\w+')
#     hyphenated_word=tokenizer.tokenize(line)
# #     if len(hyphenated_word)>0:print(hyphenated_word)
#     for word in hyphenated_word:
#         line=line.replace(word,word.replace('\'',' '))

    line=line.replace('y2k','the year 2000')
    word_lst=line.split()
    clean_lst=[]
    for idx in range(0,len(word_lst)):
        if word_lst[idx] not in punctuation and word_lst[idx] not in stopwordList:
#             clean_lst.append(word_lst[idx])
            dual_words=change_dual_words(word_lst[idx])
            if dual_words[1]=='<allclear>':
                    clean_lst.append(dual_words[0])
            else:
                    clean_lst.append(dual_words[0])
                    clean_lst.append(dual_words[1])
#     processed_lst.append('</s>')
    return(str(' ').join(clean_lst))

<h3> Change incorrect words and remove numbers</h3>

In [0]:
def correct_words(line):
    pattern = re.compile(r"(.)\1{2,}")
    line=pattern.sub(r"\1\1",line)
    line=line.replace('( see ratings chart below )','')
    line = ''.join([x if x in string.printable else '' for x in line])
        
    alphanum=re.findall(r'\(\s*\d*/10\s*\)|\d/10',line)
    for rating in alphanum:
        rates=re.findall(r'\d+/',rating)
        try:
            line=line.replace(rating,str(rates[0][:-1])+'R')
        except IndexError as IE:
            None
 
    alphanum=set(re.findall(r'\b\d+\s+bucks\b|\$\d+',line))
    for money in set(alphanum):
        line=line.replace(money,'money')
            
    alphanum=re.findall(r'\b\d+\'s\b|\b\d{2}s\b|\b\d{4}s\b',line)
    for year in alphanum:
        if len(year[:-1].strip("'"))==2:
            ryear='19'+year[:-1].strip("'")#+'Y'
        else:
            ryear=year[:-1].strip("'")#+'Y'
        if int(ryear)<=2019:
            rstr='past years'
        else:
            rstr='in future'
        line=line.replace(year,rstr)

    alphanum=[x.strip().strip('/').strip('-') for x in re.findall(r'\b\d{4}[^RY]?\b',line)]
    for year in set(alphanum):
        if int(year.strip())<=2019:
            rstr='past years'
        else:
            rstr='in future'
        line=line.replace(year.strip(),rstr)

    alphanum=set(re.findall(r'\b\d+\s+minutes\b',line))
    for time in set(alphanum):
        line=line.replace(time,'minutes')
        
    alphanum=set(re.findall(r'\b\d+hr\b',line))
#     if len(alphanum)>0:print(alphanum)
    for time in set(alphanum):
        line=line.replace(time,'hour')
        
    alphanum=re.findall(r'\b\d+th\b|\b\d+st\b',line)
#     if len(alphanum)>0:print(alphanum)
    for digi in set(alphanum):
        line=line.replace(digi,'nth')  
        
    alphanum=re.findall(r'\b\d+[^RY]\b|\b\d+\b',line)
    for digi in set(alphanum):
        line=line.replace(str(digi),'')
        
    return(line)

<h3>Main calls for Data Collection, Data Clean UP and Data Engineering</h3>

In [0]:
def get_data():
  movie_reviews_data=[]
  for x in movie_reviews.fileids():
      movr_fil=movie_reviews.open(x)
      if x.startswith('neg'):
          review=0
      else:
          review=1
      for line in movr_fil.readlines():
          if len(line.split())<5:
              continue
#         line=correct_words(line)    
#         movie_reviews_data.append([cleanLine(line.strip()),review])
          movie_reviews_data.append([line.strip(),review])
  random.shuffle(movie_reviews_data)
  print(datetime.now(),': Data Loaded...')
  movie_reviews_df=pd.DataFrame(movie_reviews_data[:15000],columns=['COMMENTS','TAG'])
  movie_reviews_df.COMMENTS=movie_reviews_df.COMMENTS.apply(lambda line:correct_words(line))
  movie_reviews_df.COMMENTS=movie_reviews_df.COMMENTS.apply(lambda line:cleanLine(line))
  print(datetime.now(),': Data Cleaned...')
  spell=SpellChecker()
  mispelled=('absolete','absoloute', 'absoltuely','amblyn','american','opportunety','ambivlaent','alzhiemer','alchoholic','aggressivelly','abundence','accomodates','actualy','adandon','admiting','admitedly','aformentioned','afterword','agee')
  for word in mispelled:
      movie_reviews_df.COMMENTS=movie_reviews_df.COMMENTS.str.replace(word,spell.correction(word))
  print(datetime.now(),': Misspelled words corrected...')
  return movie_reviews_df

def lemmetize_data(movie_reviews_df):
  stopword = stopwords.words('english')
  wnl = WordNetLemmatizer()
  movie_reviews_df.COMMENTS=movie_reviews_df.COMMENTS.str.split().apply(lambda word_lst: str(' ').join([wnl.lemmatize(word) if word not in stopword else word for word in word_lst]))
  print(datetime.now(),': Words Lemmatized..')
  return movie_reviews_df
# snowball_stemmer = SnowballStemmer('english')
# movie_reviews_df.COMMENTS=movie_reviews_df.COMMENTS.apply(lambda line: str(' ').join([snowball_stemmer.stem(word) if word not in stopword else word for word in line.split()]))
# print('Words Stemmed..')

def get_unigrams(movie_reviews_df):
  stopword = stopwords.words('english')
  stopword.extend(['(',')',"''",',','``','-','.','2','5','9','26'])
  stopwordList=set(stopword)
# tot_word_lst=[word for word_lst in movie_reviews_df.COMMENTS.str.split() for word in word_lst if word not in stopwordList]
# print(datetime.now(),": Tot no of words: ",len(tot_word_lst))
  unigrams=movie_reviews_df.COMMENTS.str.lower().str.split(expand=True).stack().unique()
  for x in stopwordList:
      unigrams=np.delete(unigrams,np.argwhere(unigrams==x))
  print(datetime.now(),": Length of unigrams: ", unigrams.shape[0])
  print(datetime.now(),': Unigrams Collected..')
  return unigrams

In [0]:
movie_reviews_df=lemmetize_data(get_data())
unigrams=get_unigrams(movie_reviews_df)

2019-10-12 18:31:53.312087 : Data Loaded...
2019-10-12 18:31:55.906790 : Data Cleaned...
2019-10-12 18:31:57.280504 : Misspelled words corrected...
2019-10-12 18:32:00.111678 : Words Lemmatized..
2019-10-12 18:32:00.785749 : Length of unigrams:  19546
2019-10-12 18:32:00.786374 : Unigrams Collected..


In [0]:
def calculate_precision(prediction, actual):
    predictions = list(prediction)
    actuals= list(actual)
    correct_labels = [predictions[i]  for i in range(len(predictions)-1) if actuals[i] == predictions[i]]
    precision = float(len(correct_labels))/float(len(predictions))
    return precision

In [0]:
def train_models(training_features,training_labels,test_features,test_labels):
    idx=len(training_chart)-1
    # Naive Bayes Classifier 
    nb_classifier = MultinomialNB().fit(training_features,training_labels) #training process
    predictions = nb_classifier.predict(test_features)
    print("Precision of NB classifier is")
    precision = calculate_precision(predictions,test_labels)
    print('NB:',precision)
    training_chart[idx].append(str(precision))
    # SVM Classifier
    svm_classifier = LinearSVC(penalty='l2', C=0.01).fit(training_features,training_labels)
    print("Precision of linear SVM classifier is:")
    predictions = svm_classifier.predict(test_features)
    precision = calculate_precision(predictions,test_labels)
    print("SVM:" + str(precision))
    training_chart[idx].append(str(precision))
    #Decision Tree
    dec_classifier = DecisionTreeClassifier(random_state=0).fit(training_features,training_labels)
    print("Precision of Decision Tree classifier is:")
    predictions = dec_classifier.predict(test_features)
    precision = calculate_precision(predictions,test_labels)
    print("DTC:" + str(precision))
    training_chart[idx].append(str(precision))
  # Logistic Regression
    logReg = LogisticRegression().fit(training_features,training_labels)
    print("Precision of Logistic Regression is:")
    predictions = logReg.predict(test_features)
    precision = calculate_precision(predictions,test_labels)
    print("LR:" + str(precision))
    training_chart[idx].append(str(precision))

In [0]:
print('Getting Features...')
unigram_data=movie_reviews_df.COMMENTS.str.lower().apply(lambda x: [1 if gram in x else 0 for gram in unigrams]).apply(pd.Series)
print('Spliting data to test train....')
training_chart=[]
training_chart.append(['Unigram','OneHot'])
xtrain,xtest,ytrain,ytest=train_test_split(unigram_data,movie_reviews_df.TAG,test_size=0.30,random_state=0)
print('Model Training in progress...')
train_models(xtrain,ytrain,xtest,ytest)

Getting Features...
Spliting data to test train....
Model Training in progress...
Precision of NB classifier is
NB: 0.6391111111111111
Precision of linear SVM classifier is:
SVM:0.6164444444444445
Precision of Decision Tree classifier is:
DTC:0.5333333333333333


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Precision of Logistic Regression is:
LR:0.6273333333333333


In [0]:
unigram_data=''
print('Getting Features....')
unigram_data=movie_reviews_df.COMMENTS.str.lower().apply(lambda x: [x.count(gram) if gram in x else 0 for gram in unigrams]).apply(pd.Series)
print('Spliting data to test train....')
training_chart.append(['Unigram','Frequency'])
xtrain,xtest,ytrain,ytest=train_test_split(unigram_data,movie_reviews_df.TAG,test_size=0.30,random_state=0)
print('Model Training in progress...')
train_models(xtrain,ytrain,xtest,ytest)
training_chart_df=pd.DataFrame(training_chart,columns=['Features','FeatureWeight','NB','SVM','DTC','LR'])
training_chart_df

Getting Features....
Spliting data to test train....
Model Training in progress...
Precision of NB classifier is
NB: 0.6348888888888888
Precision of linear SVM classifier is:
SVM:0.6217777777777778
Precision of Decision Tree classifier is:
DTC:0.536


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Precision of Logistic Regression is:
LR:0.6277777777777778


,Features,FeatureWeight,NB,SVM,DTC,LR
0,Unigram,OneHot,0.6391111111111111,0.6164444444444445,0.5333333333333333,0.6273333333333333
1,Unigram,Frequency,0.6348888888888888,0.6217777777777778,0.536,0.6277777777777778


In [0]:
import math
from textblob import TextBlob as tb

def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.words)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [0]:
# bloblist=[tb(line) for line in movie_reviews_df.COMMENTS.str.lower()]
# unigram_data=movie_reviews_df.COMMENTS.str.lower().apply(lambda x: [x.count(gram) if gram in x else 0 for gram in unigrams]).apply(pd.Series)

movie_reviews_df.COMMENTS.str.lower().apply(lambda line:tb(str(line)))
# document3 = tb("""The Colt Python is a .357 Magnum caliber revolver formerly
# manufactured by Colt's Manufacturing Company of Hartford, Connecticut.
# It is sometimes referred to as a "Combat Magnum".[1] It was first introduced
# in 1955, the same year as Smith &amp; Wesson's M29 .44 Magnum. The now discontinued
# Colt Python targeted the premium revolver market segment. Some firearm
# collectors and writers such as Jeff Cooper, Ian V. Hogg, Chuck Hawks, Leroy
# Thompson, Renee Smeets and Martin Dougherty have described the Python as the
# finest production revolver ever made.""")
# document3

0        (i, n,  , t, h, e,  , w, e, a, k, e, s, t,  , ...
1        (b, e, a, t, t, y,  , p, l, a, y,  , i, n, c, ...
2        (s, e, t, h, e,  , i, s,  , p, r, o, u, d,  , ...
3        (m, c, f, a, r, l, a, n, e,  , c, r, e, a, t, ...
4        (w, o, r, s, t,  , o, f,  , a, l, l,  , g, o, ...
5        (t, h, e,  , s, e, a, s, o, n, e, d,  , c, a, ...
6        (u, n, f, o, r, t, u, n, a, t, e, l, y,  , m, ...
7        (b, u, t,  , i, t,  , c, o, u, l, d, n,  , t, ...
8        (s, p, e, a, k, i, n, g,  , o, f,  , a, c, t, ...
9        (t, h, e, y,  , h, a, v, e,  , s, o,  , m, u, ...
10       (r, a, r, e, l, y,  , a, n, y, m, o, r, e,  , ...
11       (t, h, i, s,  , i, s,  , p, r, e, t, t, y,  , ...
12       (i, n,  , t, h, e,  , m, i, d, s, t,  , o, f, ...
13       (a,  , r, o, b, o, t, i, c,  , b, o, y,  , n, ...
14       (t, h, e,  , f, i, l, m,  , i, s,  , a,  , t, ...
15       (h, e,  , v, i, s, i, t,  , a, n, d,  , s, o, ...
16       (w, e,  , d, o,  , n, o, t,  , r, e, a, l, l, .

## Training Classifier

In [0]:
# Naive Bayes Classifier 
from sklearn.naive_bayes import MultinomialNB
nb_classifier = MultinomialNB().fit(training_features,training_labels) #training process
predictions = nb_classifier.predict(test_features)
print("Precision of NB classifier is")
precision = calculate_precision(predictions,test_gold_labels)
print(precision)

In [0]:
# SVM Classifier
#Refer to : http://scikit-learn.org/stable/modules/svm.html
from sklearn.svm import LinearSVC
svm_classifier = LinearSVC(penalty='l2', C=0.01).fit(training_features,training_labels)
print("Precision of linear SVM classifier is:")
predictions = svm_classifier.predict(test_features)
precision = calculate_precision(predictions,test_gold_labels)
print("Test data\t" + str(precision))


In [0]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
dec_classifier = DecisionTreeClassifier(random_state=0).fit(training_features,training_labels)
print("Precision of Decision Tree classifier is:")
predictions = dec_classifier.predict(test_features)
precision = calculate_precision(predictions,test_gold_labels)
print("Test data\t" + str(precision))


#### Add sentiment scores from sentiwordnet, here we take the average sentiment scores of all words 

In [0]:
def get_senti_wordnet_features(data):
    fet_vec_all = []
    for tup in data:
        sent = tup[0].lower()
        words = sent.split()
        pos_score = 0
        neg_score = 0
        for w in words:
            senti_synsets = swn.senti_synsets(w.lower())
            for senti_synset in senti_synsets:
                p = senti_synset.pos_score()
                n = senti_synset.neg_score()
                pos_score+=p
                neg_score+=n
                break #take only the first synset (Most frequent sense)
        fet_vec_all.append([float(pos_score),float(neg_score)])
    return fet_vec_all

#### Merge the two scores ####

In [0]:
def merge_features(featureList1,featureList2):
    # For merging two features
    if featureList1==[]:
        return featureList2
    merged = []
    for i in range(len(featureList1)):
        m = featureList1[i]+featureList2[i]
        merged.append(m)
    return merged

In [0]:
training_unigram_features = get_unigram_features(training_data,unigrams) # vocabulary extracted in the beginning
print(training_unigram_features)

In [0]:
training_swn_features = get_senti_wordnet_features(training_data)
print(training_swn_features)

In [0]:
training_features = merge_features(training_unigram_features,training_swn_features)

In [0]:
print(training_features[:10])

In [0]:
test_unigram_features = get_unigram_features(test_data,unigrams)
test_swn_features=get_senti_wordnet_features(test_data)
test_features= merge_features(test_unigram_features,test_swn_features)

#training_labels = get_lables(training_data)
#test_gold_labels = get_lables(test_data)

In [0]:
#For Mini-Project
#1. Preprocessing : improve with lemma, stemming, POS, bigram
#2. Feature scores : one/hot encoding (0/1), frequency, TF-IDF, probabity 
#3. Clasifiers: (NB, SVM, DT)
#4. Normalization : having uniform form of words throught 
#5. Compare with VADER : https://www.geeksforgeeks.org/python-sentiment-analysis-using-vader/